In [486]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
import nltk
import difflib
from sklearn.decomposition import LatentDirichletAllocation

pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [634]:
def show_card(name=None,show=False):
    if name is None:
        card_name = input('Enter the name of a commander: ').strip()
        if card_name.lower()=='exit':
            return
    else:
        card_name = name
    try:
        card = commanders[commanders.index.str.lower()==card_name.lower()]
        response = requests.get(card.image_uris.tolist()[0]['normal'])
    except:
        closest = difflib.get_close_matches(card_name,commanders.index.tolist(),len(commanders.index.tolist()),0)[0]
        print(f"Couldn't find {card_name}. Instead showing {closest}.")
        card = commanders[commanders.index==closest]
        response = requests.get(card.image_uris.tolist()[0]['normal'])
    img = Image.open(BytesIO(response.content))
    if show==True:
        display(img)
    return card.image_uris.tolist()[0]['normal']

# Load cards and drop columns

In [3]:
commanders = pd.read_json('Data/scryfall-default-cards.json')
commanders.drop(columns=['all_parts','arena_id','artist_ids','booster','border_color','card_back_id','card_faces',
                       'collector_number','digital','flavor_text','foil','frame','frame_effects','full_art',
                       'hand_modifier','id','illustration_id','life_modifier','mtgo_foil_id','mtgo_id',
                       'multiverse_ids','nonfoil','object','oracle_id','printed_name','prints_search_uri',
                       'promo','promo_types','rarity','related_uris','rulings_uri','scryfall_set_uri','set_type',
                       'set_uri','story_spotlight','uri','variation','variation_of','watermark'],inplace=True)

In [4]:
commanders.columns

Index(['artist', 'cmc', 'color_identity', 'color_indicator', 'colors',
       'edhrec_rank', 'games', 'highres_image', 'image_uris', 'lang', 'layout',
       'legalities', 'loyalty', 'mana_cost', 'name', 'oracle_text',
       'oversized', 'power', 'preview', 'printed_text', 'printed_type_line',
       'released_at', 'reprint', 'reserved', 'scryfall_uri', 'set', 'set_name',
       'set_search_uri', 'tcgplayer_id', 'textless', 'toughness', 'type_line'],
      dtype='object')

**Only include legendary creatures**

In [5]:
commanders = commanders[commanders.type_line.str.contains('Legendary Creature')]

**Commander Legal only**

In [6]:
commanders.legalities = [card['commander'] for card in commanders.legalities]
commanders = commanders[commanders.legalities=='legal']

**Commander is meant to be a fun and social format, so drop digital-exclusive cards**

In [7]:
commanders.games = ['paper' in card for card in commanders.games]
commanders = commanders[commanders.games]

In [8]:
commanders = commanders[commanders.layout=='normal']

In [9]:
commanders = commanders[~commanders.oracle_text.str.contains('Partner')]

**Dataset includes reprints as a separate row, so drop those to have a unique list**

In [10]:
commanders.drop_duplicates('name','last',inplace=True)

**Set the index of my DataFrame to the card name**

In [11]:
commanders.set_index('name',inplace=True)

In [14]:
commanders.shape

(786, 31)

In [12]:
show_card()

Enter the name of a commander: exit


## LDA to find topics of commanders

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
from nltk.corpus import stopwords

In [369]:
stop = list(set(stopwords.words('english')))
names = [card_name.lower().replace(',','').split() for card_name in commanders.index]
for name in names:
    stop.extend(name)
stop += ['gain','give','each','among','least','less','choice','until','end','start','beginning','upkeep','may',
         'put','under','whenever','except','permanent','player','cost','turn','gets','get','pay','deals','control',
         'controls','color','order','lose','battlefield','target','spell','card','equal','ability','activate',
         'next','step','cast','owner','opponent','time','would','instead','dealt','number','onto','though','long',
         'choose','target','converted','mana']
stop = list(set(stop))

In [370]:
texts = [text.strip().lower() for text in commanders.oracle_text]

In [371]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=20, stop_words=stop, analyzer='word')
tf = tf_vectorizer.fit_transform(texts)
tf_feature_names = tf_vectorizer.get_feature_names()

In [372]:
tf

<786x67 sparse matrix of type '<class 'numpy.int64'>'
	with 3832 stored elements in Compressed Sparse Row format>

In [373]:
lda=LatentDirichletAllocation(n_components=10,random_state=0,verbose=1,n_jobs=-1,max_iter=10)

In [374]:
lda.fit(tf)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=1)

In [375]:
def display_topics(model, feature_names, no_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append({topic_idx:[feature_names[i] for i in topic.argsort()[:-no_top_words-1:-1]]})
        print ("Topic %d:" % (topic_idx),end='\t')
        print ("\n\t\t".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]),end='\n\n')
    return topics

topic_words = display_topics(lda, tf_feature_names, 10)

Topic 0:	creature
		create
		token
		white
		green
		sacrifice
		artifact
		flying
		enters
		tokens

Topic 1:	creature
		counter
		counters
		return
		graveyard
		another
		dies
		enters
		flying
		sacrifice

Topic 2:	draw
		land
		discard
		cards
		loses
		creature
		flying
		enters
		return
		another

Topic 3:	cards
		graveyard
		library
		instant
		sorcery
		top
		costs
		paying
		copy
		flying

Topic 4:	damage
		combat
		flying
		creature
		vigilance
		trample
		attack
		destroy
		untap
		without

Topic 5:	power
		creature
		gains
		trample
		creatures
		enters
		additional
		haste
		attacks
		another

Topic 6:	creature
		creatures
		blocked
		spells
		attacking
		attacks
		becomes
		opponents
		tapped
		blocking

Topic 7:	strike
		vigilance
		creature
		flying
		sacrifice
		haste
		creatures
		counter
		attacks
		unless

Topic 8:	toughness
		power
		tap
		destroy
		add
		counter
		creatures
		cards
		enters
		indestructible

Topic 9:	library
		top
		shuffle
		search
		creature
		

In [622]:
cmdr_topics = pd.DataFrame(lda.transform(tf))
cmdr_topics.index = commanders.index
cmdr_topics.columns = ['Token Makers','Graveyard Counters','Hand Manipulation','Library Casters','Face is the Place',
                       'Big Creatures','Combat Experts','Jack of All Trades','Persistent Commanders','Library Searchers']

In [623]:
cmdr_topics.sort_values('Face is the Place',ascending=False)

,Token Makers,Graveyard Counters,Hand Manipulation,Library Casters,Face is the Place,Big Creatures,Combat Experts,Jack of All Trades,Persistent Commanders,Library Searchers
name,,,,,,,,,,
"Oros, the Avenger",0.016669,0.016670,0.016668,0.016667,0.849988,0.016668,0.016669,0.016668,0.016667,0.016667
"O-Kagachi, Vengeful Kami",0.016667,0.016668,0.016667,0.016675,0.849984,0.016670,0.016667,0.016668,0.016667,0.016667
Angus Mackenzie,0.020000,0.020000,0.020000,0.020000,0.820000,0.020000,0.020000,0.020000,0.020000,0.020000
Medomai the Ageless,0.020000,0.020001,0.020001,0.020000,0.819994,0.020000,0.020000,0.020003,0.020000,0.020000
"Numot, the Devastator",0.020000,0.020001,0.020001,0.020000,0.819984,0.020000,0.020000,0.020001,0.020010,0.020002
"Palladia-Mors, the Ruiner",0.020000,0.020001,0.020001,0.020001,0.819982,0.020005,0.020000,0.020009,0.020000,0.020000
"Drakuseth, Maw of Flames",0.020000,0.020001,0.020001,0.020003,0.819975,0.020004,0.020006,0.020008,0.020002,0.020001
"Radiant, Archangel",0.020004,0.020005,0.020002,0.020001,0.819973,0.020002,0.020003,0.020008,0.020000,0.020002
"Ruric Thar, the Unbowed",0.020000,0.020000,0.020000,0.020002,0.819973,0.020004,0.020006,0.020013,0.020002,0.020001


In [599]:
testing = cmdr_topics.copy()

In [602]:
(testing['Token Makers']-1)+(testing['Blockers']-1)

name
Syr Gwyn, Hero of Ashvale         -1.959987
Korvold, Fae-Cursed King          -1.977775
Chulane, Teller of Tales          -1.971422
Alela, Artful Provocateur         -1.565809
Kenrith, the Returned King        -1.977774
Grumgully, the Generous           -1.959995
Yorvo, Lord of Garenbrig          -1.878083
Syr Faren, the Hengehammer        -1.640760
Questing Beast                    -1.824003
Torbran, Thane of Red Fell        -1.933333
Syr Carah, the Bold               -1.971429
Syr Konrad, the Grim              -1.983331
Rankle, Master of Pranks          -1.971424
Ayara, First of Locthwain         -1.977774
Syr Elenora, the Discerning       -1.686940
Gadwick, the Wizened              -1.966665
Emry, Lurker of the Loch          -1.981817
Syr Alin, the Lion's Claw         -1.949985
Linden, the Steadfast Queen       -1.638864
Volrath, the Shapestealer         -1.638031
Tahngarth, First Mate             -1.209242
Sevinne, the Chronoclasm          -1.971427
Rayami, First of the Fallen

In [620]:
(testing[['Token Makers','Blockers']]-0.5).abs().sum(axis=1).sort_values()

name
Marrow-Gnawer                      0.080008
Commander Greven il-Vec            0.088907
Jedit Ojanen of Efrava             0.127180
Irini Sengir                       0.160003
Geist of Saint Traft               0.163806
Kazuul, Tyrant of the Cliffs       0.179246
Ib Halfheart, Goblin Tactician     0.244172
Kaysa                              0.266684
Meng Huo, Barbarian King           0.266684
Jacques le Vert                    0.266684
Brimaz, King of Oreskos            0.269267
Emmara, Soul of the Accord         0.349861
Eight-and-a-Half-Tails             0.359215
Crovax, Ascendant Hero             0.360007
Kari Zev, Skyship Raider           0.388125
Temmet, Vizier of Naktamun         0.392648
Tolsimir Wolfblood                 0.412411
Najeela, the Blade-Blossom         0.421434
Gyrus, Waker of Corpses            0.426038
Ghired, Conclave Exile             0.462590
Ur-Drago                           0.479047
Gosta Dirk                         0.479047
Inalla, Archmage Ritualist 

In [615]:
testing.loc[['Isamaru, Hound of Konda']]

,Token Makers,Graveyard Counters,Hand Manipulation,Library Casters,Brawlers,Big Creatures,Blockers,Jack of All Trades,Persistent Commanders,Library Searchers
name,,,,,,,,,,
"Isamaru, Hound of Konda",0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [487]:
def showImagesHorizontally(list_of_names):
    fig = figure()
    number_of_names = len(list_of_names)
    for i in range(number_of_names):
        a=fig.add_subplot(1,number_of_names,i+1)
        image = imread(list_of_files[i])
        imshow(image)
        axis('off')

In [637]:
def recommend_by_topic():
    topics = cmdr_topics.columns
    print("Enter number(s) of archtypes you like separated by spaces:")
    for topic_idx in range(len(topics)):
        print(f"\t{topic_idx+1}. {topics[topic_idx]}")
    likes = input().split()
    try:
        likes = list(map(int,likes))
    except:
        print("Non-digit entered.")
        return
    likes = list(set(likes))
    if len(likes)==0:
        return
    if min(likes) < 1 or max(likes) > len(topics):
        print('Entered a number out of range')
        return
    likes = [topics[i-1] for i in likes]
    top_recs = (cmdr_topics[likes]-(1/len(likes))).abs().sum(axis=1).sort_values().index.tolist()
#     if len(likes) == 1:
#         top_recs = cmdr_topics.sort_values(likes[0],ascending=False).index.tolist()
#     elif len(likes) > 1:
#         top_recs = pd.Series(,index=cmdr_topics.index)
    print(f"I recommend these commanders for {', '.join(likes)}")
    offset = 1
    while(1):
        for idx in range(offset,offset+5):
            print(f"\t{idx}. {top_recs[idx-1]}\n{show_card(top_recs[idx-1])}")
        move = input("(N)ext or (P)rev? ").lower().strip()
        if move == 'next' or move == 'n':
            offset += 5
        elif move == 'prev' or move == 'p':
            offset -= 5
        else:
            return
        if offset < 0 or offset+5 > len(top_recs):
            print("End of list.")
            return
    
#     scores = cmdr_topics[likes].diff(axis=1).fillna(0).abs().sum(axis=1)
#     print(scores.sort_values())

In [640]:
recommend_by_topic()

Enter number(s) of archtypes you like separated by spaces:
	1. Token Makers
	2. Graveyard Counters
	3. Hand Manipulation
	4. Library Casters
	5. Face is the Place
	6. Big Creatures
	7. Combat Experts
	8. Jack of All Trades
	9. Persistent Commanders
	10. Library Searchers
6 8
I recommend these commanders for Jack of All Trades, Big Creatures
	1. Rhonas the Indomitable
https://img.scryfall.com/cards/normal/front/4/5/453d854c-5092-4e00-a388-499353f246c3.jpg?1562909019
	2. Kolaghan, the Storm's Fury
https://img.scryfall.com/cards/normal/front/9/6/96376314-2085-4c57-861e-44d25e2d3bc8.jpg?1562827850
	3. Rafiq of the Many
https://img.scryfall.com/cards/normal/front/2/4/249c188d-c11d-46f7-a2d8-a67df4ba401b.jpg?1562701999
	4. Dragonlord Silumgar
https://img.scryfall.com/cards/normal/front/2/e/2eae504d-d4a9-440f-8ceb-df967cc50494.jpg?1562784323
	5. Tajic, Legion's Edge
https://img.scryfall.com/cards/normal/front/5/e/5e669a01-84d3-4fcc-8396-9e987bd89b4f.jpg?1538880611
(N)ext or (P)rev? 


In [588]:
def recommender():
    while(1):
        print("How would you like to be recommended a new Commander? (Enter the number)")
        print("\t1. There's a commander I liked playing before!")
        print("\t2. I want some general playstyles to chooose from.")
        print("\t3. WHAT'S COMMANDER???")
        how = input().strip()
        if how == '':
            return
        try:
            how = int(how)
            if how == 3:
                print("""   
   "Commander is an exciting, unique way to play Magic that is all about awesome 
    legendary creatures, big plays, and battling your friends in epic multiplayer 
    games! In Commander, each player chooses a legendary creature as the commander 
    of their deck. They then play with a 99-card deck that contains only cards of 
    their commander's colors. Also, other than basic lands, each deck can only use 
    one copy of any card. During the game, you can cast your commander multiple 
    times, meaning your favorite Legendary Creature can come back again and again 
    to lead the charge as you battle for victory!"
        Taken from Wizards of the Coast site.
""")
            elif how in [1,2]:
                break
            else:
                print('Enter 1, 2, or 3')
        except ValueError:
            print('Enter 1, 2, or 3.')
    if how == 1:
        return "TO BE IMPLEMENTED"
    elif how == 2:
        recommend_by_topic()

In [598]:
recommender()

How would you like to be recommended a new Commander? (Enter the number)
	1. There's a commander I liked playing before!
	2. I want some general playstyles to chooose from.
	3. WHAT'S COMMANDER???
2
Enter number(s) of archtypes you like separated by spaces:
	1. Token Makers
	2. Graveyard Counters
	3. Hand Manipulation
	4. Library Casters
	5. Brawlers
	6. Big Creatures
	7. Blockers
	8. Jack of All Trades
	9. Persistent Commanders
	10. Library Searchers

